In [227]:
import numpy as np
import pandas as pd

import plotly.express as px
pd.options.plotting.backend = "plotly"
from scipy.stats import binom,beta

In [289]:
# generate the data
true_classes = np.array([.3, 1 - .3])
true_p = np.array([.1, .75])
m = 10
n = 2000
h = np.random.binomial(np.tile(m, 2), true_p, (n, 2))
# then we select one of the two rows based on
# a weighted coin toss
H = h[np.arange(n), ~np.random.choice(2, size=n, p=true_classes)]

(3,)

In [290]:
# samples
K = 400
cur_theta = np.empty(K)
cur_theta[0] = .5
cur_x = np.empty((K, n))
cur_x[0] = np.random.binomial(1, cur_theta[0], n)
cur_p = np.empty((K, 2))
cur_p[0] = np.array([.5, .5])
# draw from hidden state using
# current params and observed data
# P(X | H, \theta, p)
theta_pp = np.empty((K, 2))
theta_pp[0] = np.array([1,1])
p_pp = np.empty((K, 2, 2))
p_pp[0] = np.ones((2,2))
for i in range(1, K):
    success_probs = cur_theta[i-1] * binom.pmf(H,m,cur_p[i-1, 0])
    normalization = (success_probs + (1 - cur_theta[i - 1]) * binom.pmf(H, m, cur_p[i - 1, 1]))
    success_probs = success_probs / normalization
    cur_x[i] = np.random.binomial(np.ones(n, dtype=np.int8), success_probs)

    # compute conditional prob by exploiting
    # conjugate priors
    theta_pp[i] = theta_pp[i - 1] + np.array([cur_x[i].sum(), n - cur_x[i].sum()])
    cur_theta[i] = np.random.beta(*theta_pp[i])

    p_pp[i] = p_pp[i - 1] + np.array([[cur_x[i] @ H, cur_x[i]@(m - H)],
                      [(1 - cur_x[i])@H, (1 - cur_x[i])@(m - H)]])
    cur_p[i] = np.random.beta(p_pp[i, :, 0], p_pp[i, :, 1])
    # seems strange that when cur_p = [.5 .5] that
    # p_pp does not give 50% estimates?
    # to see the beliefs converge, do we need to resample H too?
#    print(f'delta: {((cur_theta[i] - cur_theta[i - 1])**2).round(decimals=4)}')
print(f'p: {cur_p.mean(axis=0).round(decimals=2)} \ntheta: {cur_theta.mean().round(decimals=2)}')

p: [0.71 0.1 ] 
theta: 0.33
